# 数据加载模块 (Data Loading Module)

## 📖 概述

RM-Gallery 的数据加载模块提供了统一、灵活的数据加载接口，支持从多种数据源加载数据，并将其转换为标准格式。该模块位于 `rm_gallery/core/data/load/` 目录下。

## 🏗️ 核心架构

### 设计模式
- **策略模式**: 支持不同的数据加载策略
  - `FileDataLoadStrategy`: 本地文件加载
  - `HuggingFaceDataLoadStrategy`: HuggingFace 数据集加载

- **注册模式**: 数据转换器的动态注册和管理
  - `DataConverterRegistry`: 转换器注册中心
  - 支持运行时注册新的数据格式转换器

- **模板方法模式**: 统一的数据转换接口
  - `DataConverter`: 抽象转换器基类
  - 各种具体转换器实现特定格式的转换逻辑

## 🎯 支持的数据源

### 本地文件
- **支持格式**: JSON (`.json`)、JSONL (`.jsonl`)、Parquet (`.parquet`)
- **核心特性**: 
  - 自动文件类型检测
  - 批量文件加载
  - 递归目录扫描

### Hugging Face 数据集
- **数据来源**: Hugging Face Hub 公开数据集
- **核心特性**: 
  - 流式数据加载
  - 灵活的配置选项
  - 支持数据集分片处理

## 🔧 内置数据转换器

### ChatMessageConverter (`chat_message`)
专门处理聊天对话格式的数据：
```python
{
    "messages": [
        {"role": "user", "content": "你好"},
        {"role": "assistant", "content": "您好！有什么我可以帮您的吗？"}
    ]
}
```

### GenericConverter (`*`)
通用转换器，自动识别常见字段：
```python
{
    "prompt": "用户输入",      # 支持字段: question, input, text, instruction
    "response": "模型回复"    # 支持字段: answer, output, completion
}
```

## 💡 快速上手

### 本地文件加载

In [ ]:
# 通过创建基础类实现
from rm_gallery.core.data.load.base import DataLoad
import rm_gallery.core.data     # 核心策略注册
import rm_gallery.gallery.data  # 扩展策略注册

# 配置本地文件加载参数
config = {
    "path": "/Users/xielipeng/RM-Gallery/data/reward-bench-2/data/test-00000-of-00001.parquet",
    "limit": 1000,  # 限制加载的数据条数
}

# 创建数据加载器
loader = DataLoad(
    name="rewardbench2",           # 数据集名称
    load_strategy_type="local",    # 使用本地文件加载策略
    data_source="rewardbench2",    # 指定数据源格式转换器
    config=config                  # 传入配置参数
)

# 执行数据加载
dataset = loader.run()

# 输出数据集大小
print(f"成功加载 {len(dataset)} 条数据")


成功加载 1000 条数据


In [ ]:
# 通过创建工厂函数
from rm_gallery.core.data.load.base import create_load_module
from rm_gallery.core.data.build import create_build_module

config = {
    "path": "/Users/xielipeng/RM-Gallery/data/reward-bench-2/data/test-00000-of-00001.parquet",
    "limit": 1000,  # 限制加载的数据条数
}

# 创建加载模块
load_module = create_load_module(
    name="rewardbench",
    load_strategy_type="local",
    data_source="rewardbench",
    config=config
)
# 创建完整管道
pipeline = create_build_module(
    name="load_pipeline",
    load_module=load_module
)

# 运行管道
result = pipeline.run()
print(f"成功加载 {len(result)} 条数据")


成功加载 1000 条数据


### Hugging Face 数据集加载

In [ ]:
# 通过创建基础类实现
from rm_gallery.core.data.load.base import DataLoad

# 配置 Hugging Face 数据集加载参数
config = {
    "huggingface_split": "test",        # 数据集分片（train/test/validation）
    "limit": 1000,          # 限制加载的数据条数
    "streaming": False      # 是否使用流式加载
}

# 创建数据加载器
loader = DataLoad(
    name="allenai/reward-bench-2",     # HuggingFace 数据集路径
    load_strategy_type="huggingface",  # 使用 HuggingFace 加载策略
    data_source="rewardbench2",        # 指定数据源格式转换器
    config=config                      # 传入配置参数
)

# 执行数据加载
dataset = loader.run()

# 输出数据集大小
print(f"成功从 HuggingFace 加载 {len(dataset)} 条数据")


In [ ]:
# 通过创建工厂函数
from rm_gallery.core.data.load.base import create_load_module
from rm_gallery.core.data.build import create_build_module

config = {
    "huggingface_split": "test",        # 数据集分片（train/test/validation）
    "limit": 1000,          # 限制加载的数据条数
    "streaming": False      # 是否使用流式加载
}

# 创建加载模块
load_module = create_load_module(
    name="allenai/reward-bench-2",
    load_strategy_type="huggingface",
    data_source="rewardbench",
    config=config
)
# 创建完整管道
pipeline = create_build_module(
    name="load_pipeline",
    load_module=load_module
)

# 运行管道
result = pipeline.run()
print(f"成功加载 {len(result)} 条数据")

## 📊 数据输出格式

### BaseDataSet 结构
所有加载的数据都会封装为 `BaseDataSet` 对象：
```python
BaseDataSet(
    name="dataset_name",           # 数据集名称
    metadata={                     # 元数据信息
        "source": "data_source",
        "strategy_type": "local|huggingface",
        "config": {...}
    },
    datas=[DataSample(...), ...]   # 标准化的数据样本列表
)
```

### DataSample 结构
每个数据样本都统一转换为 `DataSample` 格式：
```python
DataSample(
    unique_id="md5_hash_id",        # 数据的唯一标识符
    input=[                         # 输入消息列表
        ChatMessage(role="user", content="...")
    ],  
    output=[                        # 输出数据列表
        DataOutput(answer=Step(...))
    ],      
    source="data_source_name",      # 数据源名称
    task_category="chat|qa|instruction_following|general",  # 任务分类
    metadata={                      # 详细元数据
        "raw_data": {...},          # 原始数据
        "load_strategy": "ConverterName",  # 使用的转换器
        "source_file_path": "...",  # 源文件路径（本地文件）
        "dataset_name": "...",      # 数据集名称（HF数据集）
        "load_type": "local|huggingface"   # 加载方式
    }
)
```

## 🛠️ 自定义数据转换器

如果需要支持新的数据格式，可以按以下步骤创建自定义转换器：

### 步骤 1: 实现转换器类
在 `rm_gallery/gallery/data/load/` 目录下创建转换器文件：

```python
from rm_gallery.core.data.load.base import DataConverter, DataConverterRegistry

@DataConverterRegistry.register("custom_format")
class CustomConverter(DataConverter):
    """自定义数据格式转换器"""
    
    def convert_to_data_sample(self, data_dict, source_info):
        """
        将原始数据转换为 DataSample 格式
        
        Args:
            data_dict: 原始数据字典
            source_info: 数据源信息
        
        Returns:
            DataSample: 标准化的数据样本
        """
        # 实现具体的转换逻辑
        return DataSample(...)
```

### 步骤 2: 注册转换器
在 `rm_gallery/gallery/data/__init__.py` 中导入转换器以完成注册：

```python
from rm_gallery.gallery.data.load.custom_format import CustomConverter
```

## 📚 支持的benchmark数据集

目前已内置支持以下benchmark数据集的转换器（位于 `rm_gallery/gallery/data/load/`）：

- **rewardbench**
- **rewardbench2** 
- **helpsteer2**
- **prmbench**
- **rmbbenchmark_bestofn**
- **rmbbenchmark_pairwise**

每个数据集都有对应的专用转换器，能够正确处理其特定的数据格式和字段结构。

